# TP 1 : Coder avec la tête.

### 1. Définition du problème

Soit D un document texte (par exemple, les oeuvres complètes de Shakespeare). 

Un _mot_ est une séquence consécutive de caractères alpha-numériques, tels que "Hamlet" ou "2023". Nous traiterons les lettres majuscules comme si elles étaient minuscules. Ainsi, "Hamlet" et "hamlet" sont un seul et même mot. Les mots se terminent au premier caractère non alpha-numérique. Ainsi "aujourd'hui" est constitué de deux mots : "aujourd" et "hui".

La distribution de fréquence des mots d'un document D est la fonction qui à chaque mot $w$ associe son nombre d'occurences noté $D(w)$.

On peut voir la distribution de fréquence $D$ comme un vecteur avec une coordonnée par mot, à valeur dans $\mathbb{N}$. La norme de ce vecteur se définit habituellement 

$$ N(D) = \sqrt{D\cdot D} = \sqrt{\sum_w D(w)^2}. $$

Le produit scalaire de deux vecteurs $D$ et $D'$ est aussi défini usuellement :

$$D\cdot D' = \sum_w D(w)D'(w).$$

Enfin, l'angle entre deux vecteurs $D$ et $D'$ est défini comme suit :

$$angle(D,D') = \arccos\left(\frac{D\cdot D'}{N(D)N(D')}\right).$$

Cet angle (en radians) sera un nombre entre $0$ et $\pi/2 = 1.57079632\ldots$ puisques les vecteurs sont à coordonnées positives. Clairement, on a :

- $angle(D,D) = 0$ pour tout vecteur $D$
- $angle(D,D') = \pi/2$ si $D$ et $D'$ n'ont aucun mot en commun.

On définit la distance entre deux documents comme l'angle entre leurs vecteurs distribution de fréquence. Le problème est donc de calculer la distance entre deux documents texte.

### 2. Première version du programme

Nous codons une première version du programme.


In [8]:
import math
    # math.acos(x) is the arccosine of x.
    # math.sqrt(x) is the square root of x.

import string
    # string.join(words,sep) takes a given list of words,
    #    and returns a single string resulting from concatenating them
    #    together, separated by the string sep .
    # string.lower(word) converts word to lower-case

##################################
# Operation 1: read a text file ##
##################################
def read_file(filename):
    """ 
    Read the text file with the given filename;
    return a list of the lines of text in the file.
    """
    try:
        fp = open(filename)
        L = fp.readlines()
    except IOError:
        print ("Error opening or reading input file: ",filename)
    return L

#################################################
# Operation 2: split the text lines into words ##
#################################################
def get_words_from_line_list(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        word_list = word_list + words_in_line
    return word_list

def get_words_from_string(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    word_list = []          # accumulates words in line
    character_list = []     # accumulates characters in word
    emptysymb = ""
    for c in line:
        if c.isalnum():
            character_list.append(c)
        elif len(character_list)>0:
            word = emptysymb.join(character_list)
            word = word.lower()
            word_list.append(word)
            character_list = []
    if len(character_list)>0:
        word = emptysymb.join(character_list)
        word = word.lower()
        word_list.append(word)
    return word_list

##############################################
# Operation 3: count frequency of each word ##
##############################################
def count_frequency(word_list):
    """
    Return a list giving pairs of form: (word,frequency)
    """
    L = []
    for new_word in word_list:
        for entry in L:
            if new_word == entry[0]:
                entry[1] = entry[1] + 1
                break
        else:
            L.append([new_word,1])
    return L

###############################################################
# Operation 4: sort words into alphabetic order             ###
###############################################################
def insertion_sort(A):
    """
    Sort list A into order, in place.

    From Cormen/Leiserson/Rivest/Stein,
    Introduction to Algorithms (second edition), page 17,
    modified to adjust for fact that Python arrays use 
    0-indexing.
    """
    for j in range(len(A)):
        key = A[j]
        # insert A[j] into sorted sequence A[0..j-1]
        i = j-1
        while i>-1 and A[i]>key:
            A[i+1] = A[i]
            i = i-1
        A[i+1] = key
    return A
    
#############################################
## compute word frequencies for input file ##
#############################################
def word_frequencies_for_file(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping = count_frequency(word_list)
    insertion_sort(freq_mapping)

    print ("File",filename,":",)
    print (len(line_list),"lines,",)
    print (len(word_list),"words,",)
    print (len(freq_mapping),"distinct words")

    return freq_mapping

def inner_product(L1,L2):
    """
    Inner product between two vectors, where vectors
    are represented as alphabetically sorted (word,freq) pairs.

    Example: inner_product([["and",3],["of",2],["the",5]],
                           [["and",4],["in",1],["of",1],["this",2]]) = 14.0 
    """
    sum = 0.0
    i = 0
    j = 0
    while i<len(L1) and j<len(L2):
        # L1[i:] and L2[j:] yet to be processed
        if L1[i][0] == L2[j][0]:
            # both vectors have this word
            sum += L1[i][1] * L2[j][1]
            i += 1
            j += 1
        elif L1[i][0] < L2[j][0]:
            # word L1[i][0] is in L1 but not L2
            i += 1
        else:
            # word L2[j][0] is in L2 but not L1
            j += 1
    return sum

def vector_angle(L1,L2):
    """
    The input is a list of (word,freq) pairs, sorted alphabetically.

    Return the angle between these two vectors.
    """
    numerator = inner_product(L1,L2)
    denominator = math.sqrt(inner_product(L1,L1)*inner_product(L2,L2))
    return math.acos(numerator/denominator)

def main1(filename_1,filename_2):
    sorted_word_list_1 = word_frequencies_for_file(filename_1)
    sorted_word_list_2 = word_frequencies_for_file(filename_2)
    distance = vector_angle(sorted_word_list_1,sorted_word_list_2)
    print ("The distance between the documents is: %0.6f (radians)"%distance)

Testons le sur quelques fichiers

In [11]:
file1 = "./data/t1.verne.txt" ## taille : 51 ko
file2 = "./data/t2.bobsey.txt" ## taille : 256 ko
file3 = "./data/t3.lewis.txt" ## taille : 1 Mo
file4 = "./data/t4.arabian.txt" ## taille : 3 Mo
file5 = "./data/t5.churchill.txt" ## taille : 9 Mo
file6 = "./data/t8.shakespeare.txt" ## taille : 5 Mo

In [14]:
main1(file3,file2)

File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
The distance between the documents is: 0.574160 (radians)


Nous observons que le calcul est vite très lent... 

### 3. Deuxième version, pour chasser le coupable.

Afin de déterminer les opérations qui prennent le plus de temps, nous ajoutons à la fin du programme la méthode de profiling qui permet de faire des rapports statistiques sur le nombre d'appels et le temps passé dans chaque sous-routine.

#### Question 1. 
_D'après vous, qui est le coupable du temps passé à calculer ?_

In [20]:
def main2(filename_1,filename_2):
    import profile
    profile.run("main1('"+filename_1+"','"+filename_2+"')")

In [21]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 0.574160 (radians)
         3352701 function calls in 33.011 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.001    0.001   32.947   16.474 3695471828.py:112(word_frequencies_for_file)
        3    0.038    0.013    0.057    0.019 3695471828.py:130(inner_product)
        1    0.000    0.000    0.057    0.057 3695471828.py:156(vector_angle)
        2    0.000    0.000    0.004    0.002 3695471828.py:16(read_file)
        1    0.004    0.004   33.009   33.009 3695471828.py:166(main1)
        2   14.612    7.306   17.715    8.858 3695471828.py:32(get_words_from_line_list)
    22663    1.660    0.000    3.104    0.000 3695471828.py:44(get_words_from_string)
        2   11.347    5.674   11.353    5.677 3695471828.py:73(count_frequ

On observe que la routine passe le plus clair de son temps (colonne tottime' dans la méthode `get_words_from_line_list`). Observons cela de plus près.

```
def get_words_from_line_list(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        word_list = word_list + words_in_line
    return word_list
```

#### Question 2.

_Qu'est-ce qui ne va pas ?_

In [22]:
def get_words_from_line_list(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

# VOTRE CODE ICI !

Et oui, la concaténation de liste est particulièrement laborieuse et prend un temps proportionnel à la somme des tailles des deux listes... Donc ajouter un à un les termes à la liste prend un temps $O(n^2)$ alors qu'avec append ou extend, on réduit drastiquement le temps de calcul.

In [23]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 0.574160 (radians)
         3375364 function calls in 15.528 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.023    0.011    2.743    1.371 2200860914.py:1(get_words_from_line_list)
        2    0.000    0.000   15.468    7.734 3695471828.py:112(word_frequencies_for_file)
        3    0.036    0.012    0.052    0.017 3695471828.py:130(inner_product)
        1    0.000    0.000    0.052    0.052 3695471828.py:156(vector_angle)
        2    0.000    0.000    0.002    0.001 3695471828.py:16(read_file)
        1    0.005    0.005   15.526   15.526 3695471828.py:166(main1)
    22663    1.445    0.000    2.710    0.000 3695471828.py:44(get_words_from_string)
        2    9.927    4.964    9.933    4.967 3695471828.py:73(count_freque

On vient de gagner la moitié du temps de calcul. 

#### Question 4. 
_Qui est le coupable suivant ? Comment l'améliorer ?_

Il s'agit de `count_frequency` qui prend plus de la moitié du temps de calcul. Pour l'instant il ressemble à ça :

In [26]:
def count_frequency(word_list):
    """
    Return a list giving pairs of form: (word,frequency)
    """
    L = []
    for new_word in word_list:
        for entry in L:
            if new_word == entry[0]:
                entry[1] = entry[1] + 1
                break
        else:
            L.append([new_word,1])
    return L

Il cherche à chaque fois dans la liste en un temps linéaire. Les dictionnaires (et leurs tables de hachage) peuvent nous aider à gérer cette situation bien plus vite.

In [33]:
def count_frequency(word_list):
    """
    Return a list giving pairs of form: (word,frequency)
    """
    
    # VOTRE CODE ICI

In [34]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 0.574160 (radians)
         3363482 function calls in 5.208 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.022    0.011    2.676    1.338 2200860914.py:1(get_words_from_line_list)
        2    0.000    0.000    5.162    2.581 3695471828.py:112(word_frequencies_for_file)
        3    0.026    0.009    0.041    0.014 3695471828.py:130(inner_product)
        1    0.000    0.000    0.041    0.041 3695471828.py:156(vector_angle)
        2    0.000    0.000    0.006    0.003 3695471828.py:16(read_file)
        1    0.004    0.004    5.207    5.207 3695471828.py:166(main1)
    22663    1.420    0.000    2.645    0.000 3695471828.py:44(get_words_from_string)
        2    2.448    1.224    2.448    1.224 3695471828.py:90(insertion_sor

Encore un facteur 3 de gagné sur le temps total !

Les deux derniers gros morceaux sont `get_words_from_string` et `insertion_sort`. Comment les améliorer ? Actuellement, `get_words_from_string` ressemble à ça.

In [35]:
def get_words_from_string(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    word_list = []          # accumulates words in line
    character_list = []     # accumulates characters in word
    emptysymb = ""
    for c in line:
        if c.isalnum():
            character_list.append(c)
        elif len(character_list)>0:
            word = emptysymb.join(character_list)
            word = word.lower()
            word_list.append(word)
            character_list = []
    if len(character_list)>0:
        word = emptysymb.join(character_list)
        word = word.lower()
        word_list.append(word)
    return word_list

On va utiliser ```translate``` qui permet de transformer à la voler les caractères selon une table de traduction que l'on peut définir comme il nous sied. Ici, on veut remplacer les signes de ponctuation par des espaces et les lettres majuscules par leurs version en minuscules.

In [46]:
intab = string.punctuation+string.ascii_uppercase
outtab = " "*len(string.punctuation)+string.ascii_lowercase

tab = str.maketrans(intab,outtab)

def get_words_from_string(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    line = line.translate(tab)
    word_list = line.split()
    return word_list

In [47]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 0.574160 (radians)
         134555 function calls in 2.589 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.022    0.011    0.106    0.053 2200860914.py:1(get_words_from_line_list)
        2    0.000    0.000    2.543    1.272 3695471828.py:112(word_frequencies_for_file)
        3    0.026    0.009    0.041    0.014 3695471828.py:130(inner_product)
        1    0.000    0.000    0.041    0.041 3695471828.py:156(vector_angle)
        2    0.000    0.000    0.003    0.001 3695471828.py:16(read_file)
        1    0.003    0.003    2.588    2.588 3695471828.py:166(main1)
        2    2.402    1.201    2.402    1.201 3695471828.py:90(insertion_sort)
        2    0.032    0.016    0.032    0.016 3755543530.py:1(count_frequency)
    2

Le dernier problème à régler est insertion_sort. Mais on sait que c'est nul !  Alors on fait un bon petit merge_sort et tout ira bien !

In [48]:
def merge_sort(A):
  
    # VOTRE CODE ICI

def merge(L,R):

    # VOTRE CODE ICI



In [49]:
#############################################
## compute word frequencies for input file ##
#############################################
def word_frequencies_for_file(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping = count_frequency(word_list)
    merge_sort(freq_mapping)

    print ("File",filename,":",)
    print (len(line_list),"lines,",)
    print (len(word_list),"words,",)
    print (len(freq_mapping),"distinct words")

    return freq_mapping


In [50]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 1.123077 (radians)
         652339 function calls (628575 primitive calls) in 0.638 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.022    0.011    0.107    0.054 2200860914.py:1(get_words_from_line_list)
  23766/2    0.041    0.000    0.451    0.226 298243687.py:1(merge_sort)
    11882    0.230    0.000    0.402    0.000 298243687.py:13(merge)
        3    0.023    0.008    0.039    0.013 3695471828.py:130(inner_product)
        1    0.000    0.000    0.039    0.039 3695471828.py:156(vector_angle)
        2    0.000    0.000    0.003    0.001 3695471828.py:16(read_file)
        1    0.004    0.004    0.637    0.637 3695471828.py:166(main1)
        2    0.033    0.016    0.033    0.016 3755543530.py:1(count_frequency)
    2266

Est-ce que vous pouvez vous passer complétement du tri ?

In [54]:
def word_frequencies_for_file(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping = count_frequency(word_list)

    print ("File",filename,":",)
    print (len(line_list),"lines,",)
    print (len(word_list),"words,",)
    print (len(freq_mapping),"distinct words")

    return freq_mapping

def inner_product(D1,D2):
    """
    Inner product between two vectors, where vectors
    are represented as dictionaries of (word,freq) pairs.

    Example: inner_product({"and":3,"of":2,"the":5},
                           {"and":4,"in":1,"of":1,"this":2}) = 14.0
    """
   
    # VOTRE CODE ICI.

### Programme final

In [56]:
import math
    # math.acos(x) is the arccosine of x.
    # math.sqrt(x) is the square root of x.

import string
    # string.join(words,sep) takes a given list of words,
    #    and returns a single string resulting from concatenating them
    #    together, separated by the string sep .
    # string.lower(word) converts word to lower-case

##################################
# Operation 1: read a text file ##
##################################
def read_file(filename):
    """ 
    Read the text file with the given filename;
    return a list of the lines of text in the file.
    """
    try:
        fp = open(filename)
        L = fp.readlines()
    except IOError:
        print ("Error opening or reading input file: ",filename)
    return L

#################################################
# Operation 2: split the text lines into words ##
#################################################
def get_words_from_line_list(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        # Using "extend" is much more efficient than concatenation here:
        word_list.extend(words_in_line)
    return word_list

intab = string.punctuation+string.ascii_uppercase
outtab = " "*len(string.punctuation)+string.ascii_lowercase

tab = str.maketrans(intab,outtab)

def get_words_from_string(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    line = line.translate(tab)
    word_list = line.split()
    return word_list

##############################################
# Operation 3: count frequency of each word ##
##############################################
def count_frequency(word_list):
    """
    Return a dictionary mapping words to frequency.
    """
    D = {}
    for new_word in word_list:
        if new_word in D:
            D[new_word] = D[new_word]+1
        else:
            D[new_word] = 1
    return D

    
#############################################
## compute word frequencies for input file ##
#############################################
def word_frequencies_for_file(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping = count_frequency(word_list)

    print ("File",filename,":",)
    print (len(line_list),"lines,",)
    print (len(word_list),"words,",)
    print (len(freq_mapping),"distinct words")

    return freq_mapping


def inner_product(D1,D2):
    """
    Inner product between two vectors, where vectors
    are represented as dictionaries of (word,freq) pairs.

    Example: inner_product({"and":3,"of":2,"the":5},
                           {"and":4,"in":1,"of":1,"this":2}) = 14.0
    """
    sum = 0.0
    for key in D1:
        if key in D2:
            sum += D1[key] * D2[key]
    return sum


def vector_angle(L1,L2):
    """
    The input is a list of (word,freq) pairs, sorted alphabetically.

    Return the angle between these two vectors.
    """
    numerator = inner_product(L1,L2)
    denominator = math.sqrt(inner_product(L1,L1)*inner_product(L2,L2))
    return math.acos(numerator/denominator)

def main1(filename_1,filename_2):
    sorted_word_list_1 = word_frequencies_for_file(filename_1)
    sorted_word_list_2 = word_frequencies_for_file(filename_2)
    distance = vector_angle(sorted_word_list_1,sorted_word_list_2)
    print ("The distance between the documents is: %0.6f (radians)"%distance)

In [57]:
main2(file2,file3)

File ./data/t2.bobsey.txt :
6667 lines,
49785 words,
3354 distinct words
File ./data/t3.lewis.txt :
15996 lines,
182355 words,
8530 distinct words
The distance between the documents is: 0.574160 (radians)
         91330 function calls in 0.164 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 4225794408.py:112(vector_angle)
        1    0.005    0.005    0.164    0.164 4225794408.py:122(main1)
        2    0.000    0.000    0.003    0.002 4225794408.py:14(read_file)
        2    0.026    0.013    0.124    0.062 4225794408.py:29(get_words_from_line_list)
    22663    0.035    0.000    0.087    0.000 4225794408.py:47(get_words_from_string)
        2    0.027    0.014    0.027    0.014 4225794408.py:63(count_frequency)
        2    0.000    0.000    0.155    0.078 4225794408.py:79(word_frequencies_for_file)
        3    0.004    0.001    0.004    0.001 4225794408.py:97(inner_product)

On a amélioré par un facteur 200 notre programme pour notre exemple. Et surtout on est passé en linéaire ! On peut désormais calculer la distance entre Churchill et Shakespeare.

In [59]:
main2(file5,file6)

File ./data/t5.churchill.txt :
189685 lines,
1717247 words,
32544 distinct words
File ./data/t8.shakespeare.txt :
124456 lines,
929462 words,
23881 distinct words
The distance between the documents is: 0.462095 (radians)
         1260548 function calls in 1.841 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.022    0.022 4225794408.py:112(vector_angle)
        1    0.044    0.044    1.834    1.834 4225794408.py:122(main1)
        2    0.000    0.000    0.023    0.011 4225794408.py:14(read_file)
        2    0.301    0.150    1.417    0.708 4225794408.py:29(get_words_from_line_list)
   314141    0.409    0.000    0.983    0.000 4225794408.py:47(get_words_from_string)
        2    0.328    0.164    0.328    0.164 4225794408.py:63(count_frequency)
        2    0.000    0.000    1.768    0.884 4225794408.py:79(word_frequencies_for_file)
        3    0.022    0.007    0.022    0.007 4225794408.py